In [5]:
# Install necessary libraries
!pip install open3d matplotlib scikit-learn hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 2.8 MB/s eta 0:00:00m eta 0:00:010:00:01


In [3]:
# Import required libraries
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import os
import hdbscan

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
# Function to get all PCD files from a directory
def load_point_cloud_files_from_directory(directory):
    # Get all PCD files in the directory
    pcd_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pcd')]
    return pcd_files
    
# Function to load point cloud dataset
def load_point_cloud_dataset(file_paths):
    dataset = [o3d.io.read_point_cloud(pcd_file) for pcd_file in file_paths]
    return dataset

# Function to load RGB-D file pairs from a directory
def load_rgbd_files_from_directory(directory, rgb_prefix="rgb", depth_prefix="depth", rgb_ext=".jpeg", depth_ext=".jpeg"):
    rgbd_files = []
    
    # Get all RGB files in the directory
    rgb_files = [f for f in os.listdir(directory) if f.startswith(rgb_prefix) and f.endswith(rgb_ext)]
    
    for rgb_file in rgb_files:
        # Assume depth file has the same number as the RGB file
        depth_file = rgb_file.replace(rgb_prefix, depth_prefix).replace(rgb_ext, depth_ext)
        
        # Check if corresponding depth file exists
        if depth_file in os.listdir(directory):
            rgbd_files.append((os.path.join(directory, rgb_file), os.path.join(directory, depth_file)))
    
    return rgbd_files
    
# Function to load RGB-D dataset and convert to point cloud
def load_rgbd_dataset(rgbd_paths, intrinsic):
    dataset = []
    for rgb_path, depth_path in rgbd_paths:
        rgb_image = o3d.io.read_image(rgb_path)
        depth_image = o3d.io.read_image(depth_path)
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb_image, depth_image)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
        dataset.append(pcd)
    return dataset

def downsample_point_cloud(pcd, voxel_size=0.2):
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
    downsampled_points = np.asarray([voxel.grid_index for voxel in voxel_grid.get_voxels()])
    downsampled_pcd = o3d.geometry.PointCloud()
    downsampled_pcd.points = o3d.utility.Vector3dVector(downsampled_points)
    return downsampled_pcd

def extract_combined_features(pcd, voxel_size=0.2):
    # Downsample the point cloud
    downsampled_pcd = downsample_point_cloud(pcd, voxel_size=voxel_size)
    
    # Extract color features (if available)
    num_points = len(downsampled_pcd.points)
    if downsampled_pcd.has_colors():
        colors = np.asarray(downsampled_pcd.colors)
    else:
        colors = np.zeros((num_points, 3))  # Placeholder for missing color data
    
    # Ensure the number of colors matches the number of points
    if colors.shape[0] != num_points:
        colors = np.zeros((num_points, 3))  # Adjust if needed

    features = colors
    print("Extracted features shape:", features.shape)
    return features


# Function to perform KMeans clustering
def perform_kmeans_clustering(features, n_clusters=10):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(features_scaled)
    
    labels = kmeans.labels_
    return labels

# Function to perform DBSCAN clustering
def perform_dbscan_clustering(features, eps=0.05, min_samples=5):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(features_scaled)
    
    return labels

def perform_hdbscan_clustering(features, min_cluster_size=100):
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
    labels = clusterer.fit_predict(features_scaled)
    
    return labels

# Function to visualize clusters by assigning colors to points in the original point cloud
def visualize_segmentation_with_pointcloud(pcd, labels):
    unique_labels = np.unique(labels)
    colors = plt.cm.get_cmap("tab20", len(unique_labels))(labels / len(unique_labels))
    
    # Assign colors to point cloud based on cluster labels
    pcd_colors = o3d.utility.Vector3dVector(colors[:, :3])
    pcd.colors = pcd_colors
    
    o3d.visualization.draw_geometries([pcd])

def plot_clusters(features, labels, title):
    plt.figure(figsize=(10, 7))
    scatter = plt.scatter(features[:, 0], features[:, 1], c=labels, cmap='Spectral')
    plt.colorbar(scatter)
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

In [ ]:
# Define the directory containing all PCD files
point_cloud_directory = "Data/pointcloud"

# Load all PCD files from the directory
point_cloud_paths = load_point_cloud_files_from_directory(point_cloud_directory)

# Load point cloud dataset
point_cloud_dataset = load_point_cloud_dataset(point_cloud_paths)
print(point_cloud_dataset[0])

# Extract features for each point cloud
for pcd in point_cloud_dataset:
    features = extract_combined_features(pcd, voxel_size=0.2)
    kmeans_labels = perform_kmeans_clustering(features, n_clusters=10)
    visualize_segmentation_with_pointcloud(pcd, kmeans_labels[:len(np.asarray(pcd.points))])
    
# Define the directory containing all RGB-D files
rgbd_directory = "Data/rgbd"

# Load all RGB-D file pairs from the directory
rgbd_paths = load_rgbd_files_from_directory(rgbd_directory)

# Define intrinsic parameters for RGB-D camera (modify as needed)
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.set_intrinsics(640, 480, 525, 525, 319.5, 239.5)  # dummy intrinsic parameters

# Load RGB-D dataset and convert to point cloud
rgbd_dataset = load_rgbd_dataset(rgbd_paths, intrinsic)
print(rgbd_dataset)
# Extract features for each RGB-D point cloud
for pcd in rgbd_dataset:
    features = extract_combined_features(pcd, voxel_size=0.000001)
    kmeans_labels = perform_kmeans_clustering(features)
    visualize_segmentation_with_pointcloud(pcd, kmeans_labels[:len(np.asarray(pcd.points))])    

PointCloud with 581936 points.
Extracted features shape: (117714, 3)


/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/base.py:1152: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/tmp/ipykernel_118097/3214884115.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab20", len(unique_labels))(labels / len(unique_labels))


[PointCloud with 840748 points.]
Extracted features shape: (269047, 3)


/home/brina/miniconda3/envs/gpu/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
